In [11]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import *
from scipy.stats import entropy as en


#accepted_words = []
#accepted_words = d1.read().split(' ')
#allowed_words = []
#allowed_words = d2.read().split(' ')
#no_of_aw = len(allowed_words)
y = np.random.rand(1)
y = np.round(y*12972)
#target= accepted_words[y[0].astype(int)]

word_len = int(input("Enter Word length(4,5,6,7): "))
        
accepted_words = []
allowed_words = []

d3 = open("count_1w.txt")
word_f = []
word_f = d3.read().split('\n')

for t in tqdm(word_f):
    temp = t.split('\t')
    if(len(temp[0]) == word_len):
        allowed_words.append(temp[0])
#allowed_words = accepted_words[:]

F = {}
for i in tqdm(word_f):
    temp = []
    temp = i.split('\t')
    if(temp[0] in allowed_words):
        F.update({temp[0]:temp[1]})
for i in tqdm(allowed_words):
    if(i not in F):
        F.update({i:1})
        
keys = list(F.keys())
#accepted_words = keys[0:3000]
        


Enter Word length(4,5,6,7): 4


100%|████████████████████████████████| 31140/31140 [00:00<00:00, 5468998.68it/s]


In [3]:
for i in F:
    F[i] = float(F[i])
#print(F)
freq_max = F[max(F, key=F.get)]
print(freq_max)
for i in F:
    F[i] = float(F[i]/freq_max)

1226734006.0


In [4]:
def cpl(x,space):
    temp_i = x
    patterns = []
    for y in space:
        #print(y)
        temp = y
        l_c = 0
        t = ['0']*word_len #change
        for j in x:
            if(j in temp):
                if(temp_i.index(j)==temp.index(j)):
                    #print(temp_i.index(j),temp.index(j))
                    t[temp.index(j)] = '2'
                elif(temp[l_c] == j):
                    t[l_c] = '2'
                else:
                    t[l_c] = '1'
                temp=temp.replace(j,'#',1)
                #temp_i = temp_i.replace(j,'#',1)

                #print(temp)
            l_c+=1
        q = ''
        for s in t:
            q += s
        if(q not in patterns):
            patterns.append(q)
    return patterns

In [5]:
def guess_pattern(x,y):
    temp_i = x
    patterns = []
        #print(y)
    temp = y
    l_c = 0
    t = ['0']*word_len #change
    for j in x:
        if(j in temp):
            if(temp_i.index(j)==temp.index(j)):
                #print(temp_i.index(j),temp.index(j))
                t[temp.index(j)] = '2'
            elif(temp[l_c] == j):
                t[l_c] = '2'
            else:
                t[l_c] = '1'
            temp=temp.replace(j,'#',1)
        temp_i = temp_i.replace(j,'#',1)

            #print(temp)
        l_c+=1
    q = ''
    for s in t:
        q += s
    if(q not in patterns):
        patterns.append(q)
    return patterns

In [6]:
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])
def eliminate_1(x,y,sample):
    w = x
    not_exist = []
    exist = ['']*word_len
    cor = ['']*word_len
    not_elm = []
    for a in x:
        if(y[x.index(a)] == '0'):
            not_exist.append(a)
            x=x.replace(a,'#',1)
            #print(x)
        elif(y[x.index(a)] == '1'):
            exist[x.index(a)] = a
            x=x.replace(a,'#',1)
            #print(x)
        else:
            cor[x.index(a)] = a
            x=x.replace(a,'#',1)
            #print(x)
    #print(not_exist,exist,cor)
    
    l_ne = []
    l_e = []
    l_c = []
    for word in sample:
        #print(word, end=' ')
        c=0
        fl=[0]*word_len
        flag = 0
        for m in cor:
            if(m != ''):
                if(m in word and word[c] == m):
                    fl[c] = 1
            c+=1
        #print(fl,end=' ')
        c = 0
        for m in cor:
            if(m != ''):
                if(fl[c] == 1):
                    flag = 1
                else:
                    flag = 0
                    break
            c+=1
        if(flag == 1):
            l_c.append(word)
    if(l_c == []):
        l_c = sample
    #print(l_c)
    
    for word in l_c:
        temp = word
        c = 0
        for t in cor:
            if(t != ''):
                temp = replace_str_index(temp,c,'#')
            c+=1
        
        #print(temp, end=' ')
        c = 0
        fl = [0]*word_len
        flag = 0
        for m in exist:
            if(m !=''):
                
                #m in temp
                if(m in temp and temp[c] != m):
                    fl[c] = 1
                    temp = temp.replace(m,'#',1)
                    #print(temp)
            c+=1
        #print(fl,end=' ')
        c = 0
        for m in exist:
            if(m != ''):
                if(fl[c] == 1):
                    #print('here')
                    flag = 1
                else:
                    flag = 0
                    break
            c+=1
        if(flag == 1):
            l_e.append(word)
            
    #removed y!='111111'
    if(l_e == []):
        l_e = l_c
        #l_e = []
    #print(l_e)
    temp = l_e
    #print(temp)
    for word in temp:
        #print(word)
        for m in not_exist:
            #print(m)
            count = 0
            count = exist.count(m) + cor.count(m)
            #print(count)
            if(m in word):
                if(count==0):
                    l_ne.append(word)
                    #print("removed cause no existence")
                    break
                elif(count>0):
                    if(word.count(m) > count):
                        #print("removed cause existence more than should be")
                        l_ne.append(word)
                        break
    #print(l_ne)

    return [x for x in l_e if not x in l_ne or l_ne.remove(x)]

In [7]:
def entropy2(sample_space):
    E = {}
    #tqdm
    for word in tqdm(sample_space):
        patterns = cpl(word,sample_space)
        #print(word)
        prob = []
        for pattern in patterns:
            sample = eliminate_1(word,pattern,sample_space)
            prob.append(len(sample)/len(sample_space))


        #entropy
        freq = F[word]
        #t = np.array(prob)*freq
        #print(t," ",type(t))
        #e = en(prob, base=2)
        #e = e*freq
        e = 0
        for i in prob:
            if(i != 0):
                e = e - i*freq*log2(i)
            else:
                e = e - i*freq*log2(i+1)
            #print("Information bit: ",e)
        E.update({word:e})
        
        
        
        
    return E

In [8]:
def get_graph(word,sample_space):
    patterns = cpl(word,sample_space)
    c=0
    for pattern in patterns:
        c+=1
        sample = eliminate_1(word,pattern,sample_space)
        temp = (len(sample)/len(sample_space))
        plt.bar(c,temp)
    plt.show()

In [8]:
E = entropy2(allowed_words)
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)

  0%|                                     | 39/31140 [01:25<18:51:50,  2.18s/it]


KeyboardInterrupt: 

In [9]:
# first guess
s = eliminate_1('trace','20200',allowed_words)
print(len(s))
E = entropy2(s)
E = dict(sorted(E.items(), key=lambda item: item[1]))
for i in E:
    print(i,E[i])
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)

57


100%|██████████████████████████████████████████| 57/57 [00:00<00:00, 967.22it/s]

tyabb 1.1548021945718492e-05
taaza 1.5233000541507612e-05
toaol 1.6389491961874794e-05
tiaki 2.0359796366621186e-05
thall 2.0963732423128094e-05
tfans 2.611452932699709e-05
tdaxp 2.7061610194644213e-05
thatz 2.8632887682058596e-05
thatd 3.0047782667336237e-05
tiamo 3.056249386637719e-05
thast 3.327164214107686e-05
tians 3.468554849563184e-05
tyana 3.471376558387288e-05
tiada 3.594964728334896e-05
thaba 3.693780372883025e-05
tsala 3.847271792271941e-05
toano 3.879054936832543e-05
toatl 4.222911279170131e-05
taaat 4.498629790899317e-05
tmail 4.627159710000832e-05
tiant 4.839366784476017e-05
thata 5.0461167293108034e-05
thatn 5.615000985287581e-05
thaks 5.688683671047995e-05
thala 5.755024697868902e-05
tials 6.151392484423656e-05
tdata 6.44718744795437e-05
thaad 6.463242798446503e-05
taang 6.801617308175651e-05
twatt 8.044032652204094e-05
tyagi 8.738314209565603e-05
toady 8.919594743412204e-05
thant 0.00010769782070206015
thaws 0.00011898284847553634
tmath 0.00012282505306480994
thali 0.0

In [10]:
# second guess
s2 = eliminate_1('thank','20220',s)
print(len(s2))
E = entropy2(s2)
E = dict(sorted(E.items(), key=lambda item: item[1]))
for i in E:
    print(i,E[i])
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)

9


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 2947.97it/s]

toano 9.914328711062918e-06
tyana 1.6124307081479877e-05
tfans 1.7424484669315025e-05
tiant 1.976124042051959e-05
tians 2.2794517900140754e-05
taang 5.0479045148138214e-05
tiana 0.00015068914535240304
twang 0.00024265433853002075
tsang 0.0005458599762464479
Maximum value: tsang


In [16]:
# third guess
s3 = eliminate_1('flat','0022',s2)
print(s3)
E = entropy2(s3)
E = dict(sorted(E.items(), key=lambda item: item[1]))
for i in E:
    print(i,E[i])
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)

['what', 'chat', 'boat', 'coat', 'stat', 'goat', 'scat', 'swat', 'itat', 'gnat', 'anat', 'bhat', 'adat', 'vsat', 'nhat', 'shat', 'isat', 'gaat', 'unat', 'ciat', 'svat', 'usat', 'buat', 'acat', 'skat', 'quat', 'saat', 'ukat', 'icat', 'khat', 'ghat', 'asat', 'csat', 'cbat', 'vcat', 'daat', 'haat', 'ssat', 'zcat', 'xcat', 'wnat', 'caat', 'ncat', 'sdat', 'ctat', 'ikat', 'dhat', 'agat', 'vhat', 'xhat', 'ccat', 'ovat', 'cnat', 'hcat', 'dnat', 'cgat', 'snat', 'naat', 'cjat', 'huat', 'cuat', 'baat', 'kuat', 'wiat', 'gcat', 'gsat', 'guat', 'kiat', 'cdat', 'diat', 'bcat', 'atat', 'kaat', 'ksat', 'nwat', 'dcat', 'sgat', 'kcat', 'wcat', 'aaat', 'dsat', 'kdat', 'suat', 'soat', 'azat', 'osat', 'htat', 'jsat', 'czat', 'abat', 'idat', 'bdat', 'ucat']


100%|██████████████████████████████████████████| 93/93 [00:00<00:00, 954.31it/s]

htat 4.239184700389173e-06
bdat 4.582001555173321e-06
aaat 4.72972348920647e-06
jsat 4.780055974611468e-06
idat 4.829859181215915e-06
azat 4.836320522396607e-06
abat 5.3232672970309444e-06
czat 5.511230568291246e-06
kdat 6.065262543260435e-06
osat 6.116207678348669e-06
nwat 6.166291443363992e-06
soat 6.598262555377005e-06
ucat 6.8372436921431715e-06
suat 7.985667203675237e-06
wcat 8.545061276040855e-06
dsat 8.627471876715485e-06
atat 9.402034238020263e-06
sgat 9.445248985541527e-06
kcat 9.488142296007318e-06
ovat 1.061154816623908e-05
dcat 1.0752202217422213e-05
kaat 1.08324083366158e-05
wiat 1.0845659729582283e-05
diat 1.1026189034505825e-05
kiat 1.106357467284442e-05
ksat 1.1616787204700025e-05
guat 1.216678211204055e-05
xhat 1.317453369149402e-05
kuat 1.4313453817615506e-05
bcat 1.4642001990913019e-05
vhat 1.6595872591117905e-05
quat 1.6731467271822682e-05
baat 1.687835630359278e-05
cdat 1.7010295887420842e-05
ikat 1.7135783727374378e-05
gsat 1.7178176826577188e-05
wnat 1.7315951915

In [ ]:
# fourth guess
s4 = eliminate_1('jolly','02022',s3)
print(len(s4))
E = entropy2(s4)
E = dict(sorted(E.items(), key=lambda item: item[1]))
for i in E:
    print(i,E[i])
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)

In [ ]:
# fifth guess
s5 = eliminate_1('pills','11111',s4)
print(len(s5))
E = entropy2(s5)
E = dict(sorted(E.items(), key=lambda item: item[1]))
for i in E:
    print(i,E[i])
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)

In [ ]:
# sixth guess
s6 = eliminate_1('bills','01111',s5)
print(len(s6))
E = entropy2(s6)
E = dict(sorted(E.items(), key=lambda item: item[1]))
for i in E:
    print(i,E[i])
fin_max = max(E, key=E.get)
print("Maximum value:",fin_max)